<a href="https://colab.research.google.com/github/vale0000/VALENTINE-DANDAJENA/blob/main/VALENTINE_DANDAJENA_R227562X.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# INSTALL LIBRARIES
# ============================================
!pip install -q pytesseract pdf2image gensim spacy scikit-learn \
               pdfminer.six bertopic transformers sentence-transformers \
               umap-learn hdbscan python-docx

# ============================================
# MOUNT GOOGLE DRIVE
# ============================================
from google.colab import drive
drive.mount('/content/drive')

# ============================================
# IMPORTS
# ============================================
import os, warnings, time, re
from pdfminer.high_level import extract_text
from pdf2image import convert_from_path
import pytesseract
from bertopic import BERTopic
from bertopic.representation import MaximalMarginalRelevance
from umap import UMAP
from hdbscan import HDBSCAN
from transformers import pipeline
from gensim.corpora.dictionary import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from docx import Document
import nltk
import spacy
import pandas as pd
warnings.filterwarnings("ignore")

# ============================================
# NLP RESOURCES
# ============================================
nltk.download('punkt')
nltk.download('stopwords')
!python -m spacy download en_core_web_sm
nlp = spacy.load("en_core_web_sm")

# ============================================
# PATH TO PDF
# ============================================
PDF_PATH = "/content/drive/MyDrive/Deep Learning notes.pdf"
if not os.path.exists(PDF_PATH):
    raise FileNotFoundError(f"PDF not found at {PDF_PATH}")
print(f"📄 PDF located at {PDF_PATH}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 13.1 MB/s eta 0:00:00
Mounted at /content/drive


/usr/local/lib/python3.12/dist-packages/hdbscan/robust_single_linkage_.py:175: SyntaxWarning: invalid escape sequence '\{'
  $max \{ core_k(a), core_k(b), 1/\alpha d(a,b) \}$.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 143.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
📄 PDF located at /content/drive/MyDrive/Deep Learning notes.pdf


In [ ]:
# ============================================
full_text = ""
try:
    print("Attempting PDF text extraction using pdfminer...")
    full_text = extract_text(PDF_PATH)
    if len(full_text.strip()) < 100:  # likely image-based
        raise ValueError("Text too short, using OCR instead")
except:
    print("⚡ Using OCR for text extraction...")
    extracted_text = []
    pages = convert_from_path(PDF_PATH, dpi=120)
    for i, page in enumerate(pages):
        text = pytesseract.image_to_string(page)
        extracted_text.append(text)
        print(f"   ✓ Page {i+1} processed")
    full_text = "\n".join(extracted_text)
print(f"✅ Text extraction complete, total characters: {len(full_text)}")

Attempting PDF text extraction using pdfminer...


✅ Text extraction complete, total characters: 5442372


In [ ]:
# BREAK INTO DOCUMENT CHUNKS
# ============================================
documents = [p.strip() for p in full_text.split('\n\n') if len(p.split()) > 30]
MAX_DOCS = 2500
if len(documents) > MAX_DOCS:
    documents = documents[:MAX_DOCS]
print(f"Using {len(documents)} documents for topic modeling.")

# ============================================
# BERTopic TOPIC MODELING
# ============================================
print("Running BERTopic...")
umap_model = UMAP(n_neighbors=12, n_components=3, min_dist=0.15, metric='cosine', random_state=12)
hdbscan_model = HDBSCAN(min_cluster_size=10, metric='manhattan', cluster_selection_method='leaf', prediction_data=True)
representation_model = MaximalMarginalRelevance(diversity=0.5)

topic_model = BERTopic(
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    low_memory=True,
    representation_model=representation_model,
    calculate_probabilities=False,
    verbose=False
)

topics, probabilities = topic_model.fit_transform(documents)

# Build topic table
topic_analysis = []
for topic_id in set(topics):
    if topic_id != -1:
        words = topic_model.get_topic(topic_id)
        keywords = ", ".join([w[0] for w in words[:6]])
        doc_count = list(topics).count(topic_id)
        topic_analysis.append({"topic_id": topic_id, "keywords": keywords, "doc_count": doc_count})
print(f"Identified {len(topic_analysis)} topics.")

Using 2500 documents for topic modeling.
Running BERTopic...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Identified 58 topics.


In [ ]:
# SUMMARIZATION
# ============================================
print("Generating document summary...")
summarizer = pipeline("summarization", model="philschmid/bart-large-cnn-samsum", device=-1)
MAX_CHARS = 650
OVERLAP = 150
chunks = [full_text[i:i+MAX_CHARS] for i in range(0, len(full_text), MAX_CHARS - OVERLAP)]
summaries = []
for chunk in chunks[:80]:
    try:
        out = summarizer(chunk, max_length=130, min_length=50, do_sample=False)[0]["summary_text"]
        summaries.append(out)
    except:
        continue
final_summary = " ".join(summaries)

# ============================================
# COHERENCE
# ============================================
texts = [doc.split() for doc in documents[:1500]]
coherence_topics = [[w[0] for w in topic_model.get_topic(tid)] for tid in topic_model.get_topics() if tid != -1]
id2word = Dictionary(texts)
coherence_model = CoherenceModel(topics=coherence_topics, texts=texts, dictionary=id2word, coherence="c_v")
coherence_score = coherence_model.get_coherence()
print(f"Coherence score: {coherence_score:.4f}")

Generating document summary...


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/300 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cpu


Coherence score: 0.4190


In [ ]:
from datetime import datetime
from docx import Document
from docx.shared import Inches
import matplotlib.pyplot as plt
import os

# --- SAVE RESULTS TO WORD DOC (Enhanced Unique Version with Charts) ---
docx_path = "/content/drive/MyDrive/NLP_SUMMARY.docx"
doc = Document()

# 0. Title with timestamp
timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
doc.add_heading(f"Deep Learning Document Analysis (Run: {timestamp})", level=0)

# 1. Summary section
doc.add_heading("1. Summary", level=1)
doc.add_paragraph(final_summary if 'final_summary' in globals() else "Summary not available.")

# 2. Topics Discovered
doc.add_heading("2. Topics Discovered", level=1)
if 'topic_analysis' in globals() and len(topic_analysis) > 0:
    for t in topic_analysis:
        doc.add_paragraph(f"Topic {t['topic_id']}: {t['keywords']} ({t['doc_count']} sections)")
else:
    doc.add_paragraph("No topics discovered.")

# 2a. Pie chart of topic distribution
if 'topic_analysis' in globals() and len(topic_analysis) > 0:
    labels = [f"Topic {t['topic_id']}" for t in topic_analysis]
    sizes = [t['doc_count'] for t in topic_analysis]
    plt.figure(figsize=(6,6))
    plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=140)
    plt.title("Topic Distribution")
    pie_chart_path = "/content/topic_distribution.png"
    plt.savefig(pie_chart_path, bbox_inches='tight')
    plt.close()
    doc.add_picture(pie_chart_path, width=Inches(5))
else:
    doc.add_paragraph("Pie chart not available.")

# 3. Topic Coherence
doc.add_heading("3. Topic Coherence", level=1)
doc.add_paragraph(f"Computed topic coherence (c_v): {coherence_score:.4f}" if 'coherence_score' in globals() else "N/A")

# 4. Detected Headings & Chapters
doc.add_heading("4. Headings & Chapters Detected", level=1)
if 'headings' in globals() and len(headings) > 0:
    for h in headings:
        doc.add_paragraph(f"- {h}")
else:
    doc.add_paragraph("No headings or chapters detected.")

# 5. Themes / Keywords
doc.add_heading("5. Themes / Top Keywords", level=1)
top_keywords = []
top_counts = []

# Get top keywords and counts for bar chart
if 'tfidf' in globals():
    try:
        top_keywords = [kw for kw in tfidf.get_feature_names_out()[:10]]
        top_counts = [round(tfidf.idf_[i], 2) for i in range(len(top_keywords))]
    except:
        top_keywords, top_counts = [], []
elif 'topic_analysis' in globals():
    # Fix: Access the 'keywords' string from the dictionary before splitting
    top_keywords = [kw['keywords'].split(',')[0].strip() for kw in topic_analysis[:10]]
    top_counts = [t['doc_count'] for t in topic_analysis[:10]]

doc.add_paragraph(", ".join(top_keywords) if top_keywords else "N/A")

# 5a. Bar chart for top keywords
if len(top_keywords) > 0:
    plt.figure(figsize=(8,4))
    plt.bar(top_keywords, top_counts, color='skyblue')
    plt.xticks(rotation=45, ha='right')
    plt.title("Top Keywords / Themes")
    plt.ylabel("Counts / Importance")
    bar_chart_path = "/content/top_keywords.png"
    plt.tight_layout()
    plt.savefig(bar_chart_path)
    plt.close()
    doc.add_picture(bar_chart_path, width=Inches(5))
else:
    doc.add_paragraph("Bar chart not available.")

# 6. Document Metadata
doc.add_heading("6. Document Metadata", level=1)
doc.add_paragraph(f"Total characters extracted: {len(full_text) if 'full_text' in globals() else 'N/A'}")
doc.add_paragraph(f"Total pseudo-sections used: {len(documents) if 'documents' in globals() else 'N/A'}")
doc.add_paragraph(f"PDF Path: {PDF_PATH if 'PDF_PATH' in globals() else 'N/A'}")
doc.add_paragraph(f"Analysis Timestamp: {timestamp}")

# Save Word document
doc.save(docx_path)
print(f"✅ Saved Word document with charts → {docx_path}")

✅ Saved Word document with charts → /content/drive/MyDrive/NLP_SUMMARY.docx
